## Quick Start: Training an IMDb sentiment model with *ULMFiT*

Let's start with a quick end-to-end example of training a model. We'll train a sentiment classifier on a sample of the popular IMDb data, showing 4 steps:

1. Reading and viewing the IMDb data
1. Getting your data ready for modeling
1. Fine-tuning a language model
1. Building a classifier

In [0]:
from fastai.text import * 

Contrary to images in Computer Vision, text can't directly be transformed into numbers to be fed into a model. The first thing we need to do is to preprocess our data so that we change the raw texts to lists of words, or tokens (a step that is called tokenization) then transform these tokens into numbers (a step that is called numericalization). These numbers are then passed to embedding layers that will convert them in arrays of floats before passing them through a model.

Steps:

1. Get your data preprocessed and ready to use,
1. Create a language model with pretrained weights that you can fine-tune to your dataset,
1. Create other models such as classifiers on top of the encoder of the language model.

To show examples, we have provided a small sample of the [IMDB dataset](https://www.imdb.com/interfaces/) which contains 1,000 reviews of movies with labels (positive or negative).

In [0]:
path = untar_data(URLs.IMDB_SAMPLE)

In [0]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


#### Get the data and create a `databunch` object to use for a language model

In [0]:
%%time

# throws `BrokenProcessPool` Error sometimes. Keep trying `till it works!
count = 0
error = True
while error:
    try: 
        # The following line throws `AttributeError: backwards` on the learning step, below
        # data_lm = TextDataBunch.from_csv(path, 'texts.csv')
        # This Fastai Forum post shows the solution:
        #      https://forums.fast.ai/t/backwards-attributes-not-found-in-nlp-text-learner/51340?u=jcatanza
        # We implement the solution on the following line:
        data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
        error = False
        print(f'failure count is {count}\n')    
    except: # catch *all* exceptions
        # accumulate failure count
        count = count + 1
        print(f'failure count is {count}')

failure count is 0

Wall time: 13 s

#### Get the data again, this time form a `databunch` object for use in a classifier model

In [0]:
%%time

# throws `BrokenProcessPool` Error sometimes. Keep trying `till it works!
count = 0
error = True
while error:
    try: 
        # Create the databunch for the classifier model
        data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=32)
        error = False
        print(f'failure count is {count}\n')    
    except: # catch *all* exceptions
        # accumulate failure count
        count = count + 1
        print(f'failure count is {count}')


failure count is 1

Wall time: 22.7 s

#### Save the `databunch` objects for later use

In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

#### Load the `databunch` objects
Note that you can load the data with different [`DataBunch`](/basic_data.html#DataBunch) parameters (batch size, `bptt`,...)

In [0]:
#bs=192
#bs=48
bs=32

In [0]:
data_lm = load_data(path, 'data_lm_export.pkl', bs=bs)
data_clas = load_data(path, 'data_clas_export.pkl', bs=bs)

### 1. Build and fine-tune a language model

We can use the `data_lm` object we created earlier to fine-tune a pretrained language model. [fast.ai](http://www.fast.ai/) has an English model with an AWD-LSTM architecture available that we can download. We can create a learner object that will directly create a model, download the pretrained weights and be ready for fine-tuning.

#### Set up to use the GPU

In [0]:
# torch.cuda.set_device(1)
torch.cuda.set_device(0)

#### Build the IMDb language model, initializing with the hpretrained weights from `wikitext-103`

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

#### Start training
By default this step unfreezes and trains the weights for the last layer. Training updates the weights to values more applicable to the language of IMDb reviews.

In [0]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.324914,3.921263,0.279464,00:09


In [0]:
language_model_learner??

In [0]:
learn.load_pretrained??

In [0]:
convert_weights??

You can use [Visual Studio Code](https://code.visualstudio.com/) (vscode - open source editor that comes with recent versions of Anaconda, or can be installed separately), or most editors and IDEs, to browse code. vscode things to know:

- Command palette (<kbd>Ctrl-shift-p</kbd>)
- Go to symbol (<kbd>Ctrl-t</kbd>)
- Find references (<kbd>Shift-F12</kbd>)
- Go to definition (<kbd>F12</kbd>)
- Go back (<kbd>alt-left</kbd>)
- View documentation
- Hide sidebar (<kbd>Ctrl-b</kbd>)
- Zen mode (<kbd>Ctrl-k,z</kbd>)

#### Unfreeze the weights and train some more

Like a computer vision model, we can then unfreeze the model and fine-tune it.
In this step we are allowing the model to update *all* the weights with values more suitable to the language of IMDb reviews. But we are mindful that the pretrained weights from wikitext-103 are likely already near their optimal values. For this we train the weights in the "earlier" layers with a much lower learning rate.

In [0]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4,1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,3.985726,3.922798,0.282494,00:12
1,3.683273,3.900530,0.281920,00:12
2,3.131985,3.939293,0.280150,00:12


#### Test the language model

To evaluate your language model, you can run the [`Learner.predict`](/basic_train.html#Learner.predict) method and specify the number of words you want it to guess.

In [0]:
learn.predict("This is a review about", n_words=10)

'This is a review about India and is about bonanza ANY DEFEATED'

In [0]:
learn.predict("In the hierarchy of horror movies this has to be near the top.", n_words=10)

'In the hierarchy of horror movies this has to be near the top. This is much better , but there are more'

Sometimes the generated text doesn't make much sense because we have a tiny vocabulary and didn't train much. But note that the model respects basic grammar, which comes from the pretrained model.

Finally we save the encoder to be able to use it for classification in the next section.

#### Save the encoder

In [0]:
learn.save('mini_imdb_language_model')
learn.save_encoder('mini_imdb_language_model_encoder')

### 2. Build a movie review classifier
We use mixed precision (`.to_fp16()`)for greater speed, smaller memory footprint, and a regularizing effect.

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5).to_fp16()
learn.load_encoder('mini_imdb_language_model_encoder')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (799 items)
x: TextList
xxbos xxmaj in theory , films should be a form of entertainment . xxmaj while this xxunk documentaries and other experimental forms of film - making ; most movies , specially genre films , must not only tell it 's story or message , they must entertain their target audience in some way . xxmaj all this just to say that in my opinion a bad movie is not a movie with low production values or low - budget , a bad movie is one that is boring . 
 
 " xxmaj hellborn " or " xxmaj asylum of the xxmaj damned " as is known in the xxup u.s. , is a bad movie simply because it is just not involving , and xxunk boring and tiresome . xxmaj while it has a very good premise , it is just poorly developed and the mediocre acting does n't make things better . xxmaj on another hands the film probably could had been a fine or even classic b - movie , but here it is just a bad attempt at film - making . 
 
 xxmaj director xxmaj philip xxup j. xxmaj jones tells the tale of xxmaj james xxmaj bishop ( xxmaj matt xxmaj stasi ) , a young xxunk resident , who just got his dream job at xxmaj st. xxmaj andrew xxmaj mental xxmaj hospital ; but the old asylum seems to hide a secret . xxmaj after the mysterious death of some patients and the constant rumors of satanic xxunk , xxmaj james decides to find out what is going on ; only to find the xxunk of his boss , xxmaj dr. xxunk ( xxmaj bruce xxmaj payne ) , who believes that xxmaj bishop is going as insane as his patients . 
 
 xxmaj while the premise is quite interesting , the execution of the film leaves a lot to be desired . xxmaj in an attempt of making a supernatural psychological thriller , xxmaj jones goes for the easy way out and makes a movie filled with every cliché of the genre . xxmaj of course , there are lots of great movies that are also filled with clichés ; but in " xxmaj hellborn " every single one is wasted and turned into a cheap jump scare to keep things moving , resulting in a boring and predictable storyline . 
 
 xxmaj the acting is quite mediocre for the most part , with one big exception : xxmaj bruce xxmaj payne gives a top - notch performance that makes the movie look xxunk of such good acting . xxmaj matt xxmaj stasi is very weak as the lead character and the rest of the cast make forgettable performances . 
 
 xxmaj despite all this flaws , one thing has to be written about " xxmaj hellborn " ; it has a visual look very good for the budget and very similar to modern day big - budget xxmaj xxunk " horror " productions . xxmaj also , the make - up and prosthetics are done very nicely and the designs for the main antagonist are quite good . xxmaj sadly , the rest of the xxmaj special xxmaj effects are awful and outdated , making a huge contrast with the make - up & prosthetics . 
 
 " xxmaj hellborn " is a movie with a few good things xxunk by its serious flaws with terrible results . xxmaj hardcore horror or b - movie fans may be interested by its premise but it is a boring and tiresome experience . 3 / 10,xxbos xxmaj in order to hold the public 's attention for three hours , we were treated not so much to a family 's romp through four generations and xxunk years of xxmaj hungarian history , as to sexual xxunk with a sister , a sister - in - law and other xxunk . xxmaj oh yes , there was also a totally gratuitous rape . xxmaj having said all this , the first story of the relationship among the children of the patriarch was fresh and sensual - thanks to xxmaj jennifer xxmaj xxunk .,xxbos xxmaj in the many films i have seen xxmaj warren xxmaj oates , i have come to a definite conclusion , here is one talented individual . i first saw xxmaj mr. xxmaj oates back in the 1960 's television series called xxmaj xxunk xxmaj burke . xxmaj from then on , i followed his career closely and felt he was destined for great roles . xxmaj that happened in 1974 , when xxmaj sam xxmaj peckinpah gave him top billing in a film ca

In [0]:
data_clas.show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxmaj this film sat on my xxmaj tivo for weeks before i watched it . i dreaded a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj yorkers . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj ronde",positive
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first stealth games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - rounded gaming experience in general . xxmaj with graphics",positive
"xxbos \n \n i 'm sure things did n't exactly go the same way in the real life of xxmaj xxunk xxmaj xxunk as they did in the film adaptation of his book , xxmaj xxunk xxmaj boys , but the movie "" xxmaj october xxmaj sky "" ( an xxunk of the book 's title ) is good enough to stand alone . i have not read xxmaj",positive
"xxbos xxmaj to review this movie , i without any doubt would have to quote that memorable scene in xxmaj tarantino 's "" xxmaj pulp xxmaj fiction "" ( xxunk ) when xxmaj jules and xxmaj vincent are talking about xxmaj mia xxmaj wallace and what she does for a living . xxmaj jules tells xxmaj vincent that the "" xxmaj only thing she did worthwhile was pilot "" .",negative


#### Train the last layer of the classifier

In [0]:
learn.fit_one_cycle(1, 1e-2)

#### Unfreeze the classifier model and fine-tune

In [0]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.592132,0.516068,0.771144,01:01
1,0.454031,0.321474,0.870647,01:01
2,0.338722,0.323645,0.850746,01:01


#### Test the classifier
We can use our model to predict on a few example of movie review-like raw text by using the [`Learner.predict`](/basic_train.html#Learner.predict) method.

Our model is 70% sure that this is a positive review.

In [0]:
learn.predict('Although there was lots of blood and violence, I did not think this film was scary enough.')

C:\Users\cross-entropy\Anaconda3\envs\fastai\lib\site-packages\fastai\torch_core.py:83: UserWarning: Tensor is int32: upgrading to int64; for better performance use int64 input
 warn('Tensor is int32: upgrading to int64; for better performance use int64 input')

(Category positive, tensor(1), tensor([0.3028, 0.6972]))

Our model is 83% sure that this is a positive review:

In [0]:
learn.predict('Not so good World War II epic film')

C:\Users\cross-entropy\Anaconda3\envs\fastai\lib\site-packages\fastai\torch_core.py:83: UserWarning: Tensor is int32: upgrading to int64; for better performance use int64 input
 warn('Tensor is int32: upgrading to int64; for better performance use int64 input')

(Category positive, tensor(1), tensor([0.1734, 0.8266]))

Bottom line: the model did not do a good job, misclassifying both reviews (with high confidence, to boot!)

#### Let's train some more, and re-try these examples to see if we can do better

In [0]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.263950,0.500555,0.825871,01:05
1,0.226157,0.297020,0.860696,01:05
2,0.163577,0.298125,0.855721,01:04


#### Re-try the above examples

In [0]:
learn.predict('Although there was lots of blood and violence, I did not think this film was scary enough.')

C:\Users\cross-entropy\Anaconda3\envs\fastai\lib\site-packages\fastai\torch_core.py:83: UserWarning: Tensor is int32: upgrading to int64; for better performance use int64 input
 warn('Tensor is int32: upgrading to int64; for better performance use int64 input')

(Category negative, tensor(0), tensor([0.9879, 0.0121]))

In [0]:
learn.predict('Not so good World War II epic film')

C:\Users\cross-entropy\Anaconda3\envs\fastai\lib\site-packages\fastai\torch_core.py:83: UserWarning: Tensor is int32: upgrading to int64; for better performance use int64 input
 warn('Tensor is int32: upgrading to int64; for better performance use int64 input')

(Category negative, tensor(0), tensor([0.7353, 0.2647]))

The model seems to have benefitted from the extra training, since it now correctly classifies both reviews as `negative`, with high confidence.

#### Make up your own text in the style of a movie review. Then use our classifier to see what it thinks!